[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/it-keeps-disconnecting/blob/main/nightly/stable_diffusion_v2_webui_colab.ipynb)

In [ ]:
%cd /content

%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

!apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev
!pip install -q xformers==0.0.20 triton==2.0.0 -U

!git clone -b v2.5 https://dagshub.com/camenduru/ui
!git clone https://huggingface.co/embed/negative /content/ui/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/ui/models/Lora/positive
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/ui/models/ESRGAN -o 4x-UltraSharp.pth
!git clone https://github.com/camenduru/sd-civitai-browser /content/ui/extensions/sd-civitai-browser
!git clone https://dagshub.com/camenduru/control /content/ui/extensions/control
!git clone https://dagshub.com/camenduru/deforum /content/ui/extensions/deforum
!git clone https://github.com/fkunn1326/openpose-editor /content/ui/extensions/openpose-editor
!git clone https://github.com/camenduru/tunnels /content/ui/extensions/tunnels
!git clone https://github.com/etherealxx/batchlinks-webui /content/ui/extensions/batchlinks-webui
%cd /content/ui
!git reset --hard
!git -C /content/ui/repositories/stable-diffusion-stability-ai reset --hard
!git -C /content/ui/extensions/deforum reset --hard

!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/resolve/main/canny-sd21-safe.safetensors -d /content/ui/extensions/control/models -o canny-sd21-safe.safetensors
!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/resolve/main/depth-sd21-safe.safetensors -d /content/ui/extensions/control/models -o depth-sd21-safe.safetensors
!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/resolve/main/hed-sd21-safe.safetensors -d /content/ui/extensions/control/models -o hed-sd21-safe.safetensors
!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/resolve/main/openpose-sd21-safe.safetensors -d /content/ui/extensions/control/models -o openpose-sd21-safe.safetensors
!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/resolve/main/scribble-sd21-safe.safetensors -d /content/ui/extensions/control/models -o scribble-sd21-safe.safetensors
!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/raw/main/cldm_v21.yaml -d /content/ui/extensions/control/models -o canny-sd21-safe.yaml
!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/raw/main/cldm_v21.yaml -d /content/ui/extensions/control/models -o depth-sd21-safe.yaml
!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/raw/main/cldm_v21.yaml -d /content/ui/extensions/control/models -o hed-sd21-safe.yaml
!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/raw/main/cldm_v21.yaml -d /content/ui/extensions/control/models -o openpose-sd21-safe.yaml
!aria2c --console-log-level=error -c -x 16 -k 1M -s 16 https://huggingface.co/ckpt/ControlNet/raw/main/cldm_v21.yaml -d /content/ui/extensions/control/models -o scribble-sd21-safe.yaml

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2/resolve/main/768-v-ema.ckpt -d /content/ui/models/Stable-diffusion -o 768-v-ema.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/stable-diffusion-2/raw/main/v2-inference-v.yaml -d /content/ui/models/Stable-diffusion -o 768-v-ema.yaml

!sed -i -e '''/from modules import launch_utils/a\import os''' /content/ui/launch.py
!sed -i -e '''/        prepare_environment()/a\        os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/ui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/ui/launch.py
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /content/ui/modules/shared.py
!sed -i -e 's/sd-webui-controlnet\.scripts/control\.scripts/g' /content/ui/extensions/deforum/scripts/deforum_helpers/deforum_controlnet.py

!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple